In [1]:
import os as os
import ctypes
import pickle
import multiprocessing as mp

import tqdm
import numpy as np
from scipy.io import loadmat
from scipy.signal import detrend
import nibabel as nib

import popeye.utilities as utils
from popeye import css
from popeye.visual_stimulus import VisualStimulus, resample_stimulus

In [2]:
f = open('css_results.pkl', 'rb')
results = pickle.load(f)

In [3]:
results

 ...]

In [4]:
len(results)

218854

In [5]:
output = [r for r in results if r is not None]

In [6]:
output

 ...]

In [7]:
len(output)

218854

In [8]:
nii = nib.load('/Users/aditya/Documents/GitHub/prf-mapping/sample_data/JC/RF1/JC_RF1_vista/bar_seq_1_ss5.nii.gz')

In [9]:
nii

In [10]:
dims = list(nii.shape)

In [11]:
dims

[128, 128, 128, 201]

In [12]:
dims = dims[0:3]

In [13]:
dims

[128, 128, 128]

In [14]:
dims.append(8)

In [15]:
estimates = np.zeros(dims)

In [16]:
estimates.shape

(128, 128, 128, 8)

In [17]:
import math

In [18]:
for fit in output:
    if not np.isnan(fit.rsquared):
        voxel_dat = list(fit.estimate)
        reshape_voxel_dat = []
        reshape_voxel_dat.append(math.atan2(voxel_dat[1], voxel_dat[0]))
        reshape_voxel_dat.append(fit.rsquared)
        reshape_voxel_dat.append(math.sqrt( voxel_dat[0] ** 2 + voxel_dat[1] ** 2))
        reshape_voxel_dat.append(voxel_dat[2])
        reshape_voxel_dat.append(voxel_dat[3])
        reshape_voxel_dat.append(voxel_dat[0])
        reshape_voxel_dat.append(voxel_dat[1])
        reshape_voxel_dat.append(voxel_dat[4])
        # print(reshape_voxel_dat)
        # print(math.atan2(voxel_dat[1], voxel_dat[0]))
        # print(math.sqrt( voxel_dat[0] ** 2 + voxel_dat[1] ** 2 ))
        # voxel_dat.append(fit.rsquared)
        reshape_voxel_dat = np.array(reshape_voxel_dat)
        # print(fit.voxel_index)
        # print(voxel_dat)
        estimates[tuple(fit.voxel_index)] = reshape_voxel_dat

In [19]:
grid_parent = nii

In [20]:
aff = grid_parent._affine
hdr = grid_parent._header
hdr.set_data_shape(dims)

In [21]:
import nibabel

In [22]:
nifti_estimates = nibabel.Nifti1Image(estimates,aff,header=hdr)

In [24]:
nib.save(nifti_estimates, "output/JC/prf_css.nii.gz")